In [13]:
import boto3
iam = boto3.client('iam')
response = iam.list_roles()
sagemaker_roles = [role for role in response['Roles'] if 'SageMaker' in role['RoleName']]
sagemaker_roles[0]['RoleName']

'AmazonSageMaker-ExecutionRole-20231030T210397'

In [14]:
from transformers import GPTQConfig

### Creating training data

In [16]:
from transformers import AutoTokenizer
model_id = 'teknium/OpenHermes-2.5-Mistral-7B'
tokenizer = AutoTokenizer.from_pretrained(model_id,trust_remote_code=True,use_fast=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [26]:
system_prompt = '''
You're an AI expert in financial and business topics designed by Insight, answering queries using relevant materials and following strict rules:
* Valid Queries: Questions about finance, business, or financially relevant entities (e.g., India, RBI, Reliance, Fed).
* Invalid Queries: Avoid questions unrelated to finance or business (e.g., Bollywood, sports, unless financially relevant), non-financial instructional queries (e.g., jokes, coding), and inappropriate content.

Response Instructions:
* If the query is valid and answerable only using references, summarize the information clearly and concisely.
* If the query is an entity like India, Supreme Court, Modi, RBI - and if the articles contain news about these entities - then you can mention that the latest news about ‘query’ by giving brief details. When the published date is old - mention the dates.
* Prioritize Indian entities in responses involving both Indian and international contexts. Clearly specify the country when referencing international entities.
* If it's a valid query but the exact answer isn't in the references, state your inability to find the exact answer and provide a brief summary of related information.
* If it's a valid query but there's no relevant information, apologize, say you can't find the answer, and note that the Insight team is working to expand knowledge.
* For invalid queries, politely decline, noting your focus on finance. You are provided with a timeless financial articles for you to share as a trivia to illustrate the kind of information you possess. If the reference is not timeless in these cases, then don't share any trivia.

General Response Guidelines:
* Maintain professionalism and respect in responses.
* Ensure grammatical accuracy and readability.
* Don't use the word 'references' in the response. You simply have knowledge or don't have the knowledge. 
* Keep responses concise (50-200 words, 3-4 sentences, max 6).
* Only provide answers found in references.
* Politely decline irrelevant queries, emphasizing financial expertise.
* Calculate dates using today's date provided in format %Y-%m-%d.
* When users ask for explicit stock recommendations or predictions, explicitly mention that 
You are provided with the following query_json: {'original_query': , 'guess_of_is_valid_query': ,'augmented_query_if_valid':}
prioritize original_query and use augmented_query_if_valid and guess_of_is_valid_query as suggestions. Don't communicate the augmented_query_if_valid at any cost to the user
|query_json|
'''

In [18]:
from ast import literal_eval

In [20]:
def parse_articles(a_lst):
    if len(a_lst) == 1:
        a = a_lst[0]
        return [a[i*24:(i+1)*24] for i in range(int(len(a)/24))]
    else:
        return a_lst

In [25]:
all_responses = pd.read_csv('gpt-4-search-responses-30-jan.csv',on_bad_lines='skip',header=None)

In [42]:
all_responses[12].iloc[2].split('|query_json|')

["\nYou're an AI expert in financial and business topics designed by Insight, answering queries using relevant materials and following strict rules:\n* Valid Queries: Questions about finance, business, or financially relevant entities (e.g., India, RBI, Reliance, Fed).\n* Invalid Queries: Avoid questions unrelated to finance or business (e.g., Bollywood, sports, unless financially relevant), non-financial instructional queries (e.g., jokes, coding), and inappropriate content.\n\nResponse Instructions:\n* If the query is valid and answerable only using references, summarize the information clearly and concisely.\n* If the query is an entity like India, Supreme Court, Modi, RBI - and if the articles contain news about these entities - then you can mention that the latest news about ‘query’ by giving brief details. When the published date is old - mention the dates.\n* Prioritize Indian entities in responses involving both Indian and international contexts. Clearly specify the country whe

In [47]:
from datasets import Dataset

In [48]:
sys.path.append("../utils")
from pack_dataset import pack_dataset

In [49]:
finetune_id = 'PurpleCherryPanda'

In [50]:
train_df = all_responses[[12,13]]
dataset = Dataset.from_pandas(train_df)

In [51]:
def format_as_prompt_response(response_row):
    response = response_row['13']
    input_prompt = response_row['12']
    sys_prompt, user_content = input_prompt.split('|query_json|')
    messages = [{"role": "system", "content": sys_prompt},
                {"role": "user", "content": user_content}]
    context_prompt = tokenizer.decode(tokenizer.apply_chat_template(messages, add_generation_prompt=True))
    prompt = context_prompt + response
    prompt = re.sub(r'\n+','\n',prompt)
    return prompt

In [52]:
# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_as_prompt_response(sample)}{tokenizer.eos_token}"
    return sample

dataset = dataset.map(template_dataset)
# tokenize dataset
dataset = dataset.map(
    lambda sample: tokenizer(sample["text"]), batched=True, remove_columns=list(dataset.features)
)
# chunk dataset
lm_dataset = pack_dataset(dataset, chunk_length=4096) # We use 4096 as the maximum length for packing
print(f"Total number of samples: {len(lm_dataset)}")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Map:   0%|          | 0/671 [00:00<?, ? examples/s]

Map:   0%|          | 0/671 [00:00<?, ? examples/s]

Chunking dataset into chunks of 4096 tokens.


Map:   0%|          | 0/671 [00:00<?, ? examples/s]

Total number of samples: 592
Total number of samples: 592


In [53]:
import sagemaker
import boto3
sess = sagemaker.Session()
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='AmazonSageMaker-ExecutionRole-20231030T210397')['Role']['Arn']
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::005418323977:role/service-role/AmazonSageMaker-ExecutionRole-20231030T210397
sagemaker bucket: sagemaker-ap-south-1-005418323977
sagemaker session region: ap-south-1


In [54]:
finetune_dataset_config = {'finetune_id': 'OpenHermes_Search_' + finetune_id,
                          'date': datetime.strftime(datetime.today(),'%Y-%m-%d'),
                          'num_datapoints': len(train_df),
                            'data_source': 'gpt4',
                          'prompt': system_prompt}

In [55]:
# save train_dataset to s3
training_input_path = f's3://{sess.default_bucket()}/fine_tuning_datasets/{finetune_dataset_config["date"]}-{finetune_dataset_config["finetune_id"]}'
lm_dataset.save_to_disk(training_input_path)

print("uploaded data to:")
print(f"training dataset to: {training_input_path}")

INFO:aiobotocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


Saving the dataset (0/1 shards):   0%|          | 0/592 [00:00<?, ? examples/s]

uploaded data to:
training dataset to: s3://sagemaker-ap-south-1-005418323977/fine_tuning_datasets/2024-02-01-OpenHermes_Search_PurpleCherryPanda


In [56]:
import pandas as pd
import numpy as np
import json

import re
from transformers import AutoTokenizer
from random import randint
import sys
sys.path.append("../utils")
from pack_dataset import pack_dataset
from datasets import Dataset

import requests

In [57]:
from huggingface_hub import HfFolder


# hyperparameters, which are passed into the training job
hyperparameters ={
  'model_id': model_id,                             # pre-trained model
  'dataset_path': '/opt/ml/input/data/training',    # path where sagemaker will save training dataset
  'num_train_epochs': 2,                            # number of training epochs
  'per_device_train_batch_size': 2,                 # batch size for training
  'gradient_accumulation_steps': 4,                 # Number of updates steps to accumulate
  'gradient_checkpointing': True,                   # save memory but slower backward pass
  'bf16': True,                                     # use bfloat16 precision
  'tf32': True,                                     # use tf32 precision
  'learning_rate': 5e-5,                            # learning rate
  'max_grad_norm': 0.3,                             # Maximum norm (for gradient clipping)
  'warmup_ratio': 0.03,                             # warmup ratio
  "lr_scheduler_type":"cosine_with_restarts",                   # learning rate scheduler
  'save_strategy': "epoch",                         # save strategy for checkpoints
  "logging_steps": 10,                              # log every x steps
  'merge_adapters': True,                           # wether to merge LoRA into the model (needs more memory)
  'use_flash_attn': True,                           # Whether to use Flash Attention
  'output_dir': '/tmp/run'                         # output directory, where to save assets during training
    
                                                    # could be used for checkpointing. The final trained
                                                    # model will always be saved to s3 at the end of training
}

if HfFolder.get_token() is not None:
    hyperparameters['hf_token'] = HfFolder.get_token() # huggingface token to access gated models, e.g. llama 2

In [58]:
from sagemaker.huggingface import HuggingFace

# define Training Job Name
job_name = f'huggingface-qlora-{hyperparameters["model_id"].replace("/","-").replace(".","-")}-{finetune_dataset_config["finetune_id"]}'

# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'run_qlora-original.py',    # train script
    source_dir           = '../utils/',      # directory which includes all the files needed for training
    instance_type        = 'ml.g5.4xlarge',   # instances type used for the training job
    instance_count       = 1,                 # the number of instances used for training
    max_run              = 6*60*60,        # maximum runtime in seconds (days * hours * minutes * seconds)
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    volume_size          = 50,               # the size of the EBS volume in GB
    transformers_version = '4.28',            # the transformers version used in the training job
    pytorch_version      = '2.0',             # the pytorch_version version used in the training job
    py_version           = 'py310',           # the python version used in the training job
    hyperparameters      =  hyperparameters,  # the hyperparameters passed to the training job
    environment          = { "HUGGINGFACE_HUB_CACHE": "/tmp/.cache" }, # set env variable to cache models in /tmp
    disable_output_compression = True         # not compress output to save training time and cost
)

In [59]:
# define a data input dictonary with our uploaded s3 uris
data = {'training': training_input_path}
# starting the train job with our uploaded datasets as input
huggingface_estimator.fit(data, wait=True)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-qlora-teknium-OpenHermes-2--2024-02-01-07-05-56-586


2024-02-01 07:05:58 Starting - Starting the training job...
2024-02-01 07:06:14 Starting - Preparing the instances for training......
2024-02-01 07:07:11 Downloading - Downloading input data...
2024-02-01 07:07:36 Downloading - Downloading the training image.....................
2024-02-01 07:11:17 Training - Training image download completed. Training in progress......bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-02-01 07:12:13,389 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-02-01 07:12:13,407 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-02-01 07:12:13,416 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-02-01 07:12:13,418 sagemaker_pytorch_container.training INFO     Invoking user training script.
2024-02-01 07:12:14,815 sagemaker-training-toolkit INFO     Installing dep

### Deployment

In [60]:
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
  "huggingface",
  version="1.1.0",
  session=sess,
)

# print ecr image uri
print(f"llm image uri: {llm_image}")

INFO:sagemaker.image_uris:Defaulting to only available Python version: py39
INFO:sagemaker.image_uris:Defaulting to only supported image scope: gpu.


llm image uri: 763104351884.dkr.ecr.ap-south-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.1-tgi1.1.0-gpu-py39-cu118-ubuntu20.04


In [61]:
import boto3
import os

# Initialize a boto3 S3 client
s3 = boto3.client('s3')

In [62]:
model_s3_path = huggingface_estimator.model_data["S3DataSource"]["S3Uri"]

In [63]:
model_s3_path

's3://sagemaker-ap-south-1-005418323977/huggingface-qlora-teknium-OpenHermes-2--2024-02-01-07-05-56-586/output/model/'

In [64]:
import json
from sagemaker.huggingface import HuggingFaceModel
model_s3_path = huggingface_estimator.model_data["S3DataSource"]["S3Uri"]
# sagemaker config
instance_type = "ml.g5.xlarge"
number_of_gpu = 1
health_check_timeout = 300

# Define Model and Endpoint configuration parameter
config = {
  'HF_MODEL_ID': "/opt/ml/model", # path to where sagemaker stores the model
  'SM_NUM_GPUS': json.dumps(number_of_gpu), # Number of GPU used per replica
  'MAX_INPUT_LENGTH': json.dumps(3584), # Max length of input text
  'MAX_TOTAL_TOKENS': json.dumps(4096), # Max length of the generation (including input text)
}

# create HuggingFaceModel with the image uri
llm_model = HuggingFaceModel(
  role=role,
  image_uri=llm_image,
  model_data={'S3DataSource':{'S3Uri': model_s3_path,'S3DataType': 'S3Prefix','CompressionType': 'None'}},
  env=config
)

In [65]:
finetune_id

'PurpleCherryPanda'

In [66]:
# Deploy model to an endpoint
# https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy
llm = llm_model.deploy(
    endpoint_name = f"OpenHermes-Search-{finetune_id.replace('_','-')}",
  initial_instance_count=1,
  instance_type=instance_type,
  container_startup_health_check_timeout=health_check_timeout, # 10 minutes to be able to load the model
)

INFO:sagemaker:Creating model with name: huggingface-pytorch-tgi-inference-2024-02-01-09-21-18-500
INFO:sagemaker:Creating endpoint-config with name OpenHermes-Search-PurpleCherryPanda
INFO:sagemaker:Creating endpoint with name OpenHermes-Search-PurpleCherryPanda


---------!

In [67]:
smr = sess.boto_session.client("sagemaker-runtime")

In [68]:
parameters = {
    "do_sample": True,
    "top_p": 0.9,
    "temperature": 0.7,
    "max_new_tokens": 512,
    "repetition_penalty": 1.03,
    "stop": ["###", "</s>", tokenizer.eos_token],
}

In [69]:
system_prompt = '''
You're an AI expert in financial and business topics designed by Insight, answering queries using relevant materials and following strict rules:
* Valid Queries: Questions about finance, business, or financially relevant entities (e.g., India, RBI, Reliance, Fed).
* Invalid Queries: Avoid questions unrelated to finance or business (e.g., Bollywood, sports, unless financially relevant), non-financial instructional queries (e.g., jokes, coding), and inappropriate content.

Response Instructions:
* If the query is valid and answerable only using references, summarize the information clearly and concisely.
* If the query is an entity like India, Supreme Court, Modi, RBI - and if the articles contain news about these entities - then you can mention that the latest news about ‘query’ by giving brief details. When the published date is old - mention the dates.
* Prioritize Indian entities in responses involving both Indian and international contexts. Clearly specify the country when referencing international entities.
* If it's a valid query but the exact answer isn't in the references, state your inability to find the exact answer and provide a brief summary of related information.
* If it's a valid query but there's no relevant information, apologize, say you can't find the answer, and note that the Insight team is working to expand knowledge.
* For invalid queries, politely decline, noting your focus on finance. You are provided with a timeless financial articles for you to share as a trivia to illustrate the kind of information you possess. If the reference is not timeless in these cases, then don't share any trivia.

General Response Guidelines:
* Maintain professionalism and respect in responses.
* Ensure grammatical accuracy and readability.
* Don't use the word 'references' in the response. You simply have knowledge or don't have the knowledge. 
* Keep responses concise (50-200 words, 3-4 sentences, max 6).
* Only provide answers found in references.
* Politely decline irrelevant queries, emphasizing financial expertise.
* Calculate dates using today's date provided in format %Y-%m-%d.
* When users ask for explicit stock recommendations or predictions, explicitly mention that 
You are provided with the following query_json: {'original_query': , 'guess_of_is_valid_query': ,'augmented_query_if_valid':}
prioritize original_query and use augmented_query_if_valid and guess_of_is_valid_query as suggestions. Don't communicate the augmented_query_if_valid at any cost to the user
|query_json|
'''

In [168]:
a,b,c,d = 'False,,,'.split(',')

In [169]:
a

'False'

In [172]:
b = 2; c = 3

In [173]:
a = {'1': b, '2': c}

In [174]:
a

{'1': 2, '2': 3}

In [175]:
b = 4

In [176]:
a

{'1': 2, '2': 3}

In [165]:
import json

In [167]:
json.load('False')

AttributeError: 'str' object has no attribute 'read'

In [149]:
all_responses[(all_responses[11]=='relevant')].sample(30)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
592,0,444,what is sensex,2023-11-02,"True,Low,False,What is Sensex",True,Low,False,What is Sensex,True,"['652fca6d1e5cc42b1b13a0ee', '6555b5084b13023f...",relevant,\nYou're an AI expert in financial and busines...,"The Sensex, or the S&P BSE Sensex, is India's ..."
63,0,240,terrie,2023-12-02,"False,,,",False,NaN,NaN,NaN,False,['656476fe96c80a0efa90fbcb'],relevant,\nYou're an AI expert in financial and busines...,"I'm sorry, but ""terrie"" is not a valid query r..."
397,0,232,ipo,2024-02-12,"True,High,False,Upcoming IPOs in India",True,High,False,Upcoming IPOs in India,True,"['65acc1006391ac19baa87097', '65813f89683f430d...",relevant,\nYou're an AI expert in financial and busines...,The IPO (Initial Public Offering) market in In...
179,0,272,michael scott,2024-02-07,"False,,,",False,NaN,NaN,NaN,False,['6564772b96c80a0efa90ff7f'],relevant,\nYou're an AI expert in financial and busines...,The query 'Michael Scott' is not related to th...
548,0,171,collapsed real estate firm unitech,2024-01-07,"True,Medium,True,Unitech real estate financial...",True,Medium,True,Unitech real estate financial news,True,"['65922acb4b25ce4814fb420a', '65a920496391ac19...",relevant,\nYou're an AI expert in financial and busines...,Unitech Advisors (India) mentioned in the cont...
95,0,518,tata motors stock outlook,2024-01-22,"True,Medium,False,Tata Motors stock analysis J...",True,Medium,False,Tata Motors stock analysis January 2024,True,"['65adda586391ac19baa871e6', '65a9e5d16391ac19...",relevant,\nYou're an AI expert in financial and busines...,Tata Motors' stock has seen fluctuations over ...
445,0,459,credit cards,2023-12-21,"True,Medium,False,Credit card industry trends ...",True,Medium,False,Credit card industry trends in India,True,"['657bc159857ad3220916009f', '6571c36725cc9a34...",relevant,\nYou're an AI expert in financial and busines...,Credit cards in India have shown substantial g...
273,0,225,coal india limited's stock,2024-01-17,"True,Medium,False,Coal India Limited's stock p...",True,Medium,False,Coal India Limited's stock performance,True,"['65316fe21e5cc42b1b142486', '65316f8d1e5cc42b...",relevant,\nYou're an AI expert in financial and busines...,Coal India's stock performance has seen slight...
371,0,263,stocks rising with green energy transition,2023-12-19,"True,Medium,False,Stocks rising with green ene...",True,Medium,False,Stocks rising with green energy transition in ...,True,"['658258d8683f430d81566015', '6570445153e8a611...",relevant,\nYou're an AI expert in financial and busines...,The transition to green energy appears to be p...
303,0,383,ulip as a strategic investment,2024-01-22,"True,Low,False,ULIP as a strategic investment ...",True,Low,False,ULIP as a strategic investment option,True,"['65683309207bb033b910eda3', '653680181e5cc42b...",relevant,\nYou're an AI expert in financial and busines...,Unit-Linked Insurance Plans (ULIP) combine ins...


In [150]:
inp = all_responses[all_responses[11]=='relevant'].loc[444][12]

In [151]:
sys_prompt, user_content = inp.split('|query_json|')
messages = [{"role": "system", "content": sys_prompt},
            {"role": "user", "content": user_content}]
context_prompt = tokenizer.decode(tokenizer.apply_chat_template(messages, add_generation_prompt=True))

In [153]:
len(tokenizer.encode(context_prompt))

6721

In [108]:
# len(' '.join(context_prompt.split(' ')[:2048]))

In [156]:
context_prompt = ' '.join(context_prompt.split(' ')[:1800])

In [157]:
len(tokenizer.encode(context_prompt))

3239

In [158]:
request = {"inputs": context_prompt, "parameters": parameters, "stream": True}

In [159]:
t = time.time()
resp = smr.invoke_endpoint_with_response_stream(
    EndpointName=f"OpenHermes-Search-PurpleCherryPanda",
    Body=json.dumps(request),
    ContentType="application/json",
)
print(f'done in {time.time() - t} seconds')
# k = resp['Body'].read()
for token in TokenIterator(resp["Body"]):
    print(token, end="")

done in 0.35460495948791504 seconds
0 AUM: ₹6929.06 Crore Expense Ratio: 1.93% Risk: High Risk Min Investment: SIP: Rs. 500 Lump Sum: Rs.5000 3-Year Return: 24.76% 5-Year Return: 17.21% 3. Nippon India Multi-Cap Fund This fund has been created to give investors the opportunity to invest in a diversified portfolio of stocks across market capitalisation classes. The fund aims to offer high returns for investors with a long-term investment horizon. It invests at least 65% of its assets in equity and equity-related instruments. It also allows investors to choose between the direct and regular plans. NAV: ₹162.90 AUM: ₹14091.596 Crore Expense Ratio: 1.88% Risk: High Risk Min Investment: SIP: Rs. 500 Lump Sum: Rs.5000 3-Year Return: 12.49% 5-Year Return: 14.71% 4. Quant Active Fund Quant Active Fund has been designed to provide investors with the opportunity to invest in a diversified portfolio of stocks across market capitalisation classes. The fund invests at least 65% of its assets in equ

In [80]:
import io
import json

class TokenIterator:
    def __init__(self, stream):
        self.byte_iterator = iter(stream)
        self.buffer = io.BytesIO()
        self.read_pos = 0

    def __iter__(self):
        return self

    def __next__(self):
        while True:
            self.buffer.seek(self.read_pos)
            line = self.buffer.readline()
            if line and line[-1] == ord("\n"):
                self.read_pos += len(line) + 1
                full_line = line[:-1].decode("utf-8")
                line_data = json.loads(full_line.lstrip("data:").rstrip("/n"))
                return line_data["token"]["text"]
            chunk = next(self.byte_iterator)
            self.buffer.seek(0, io.SEEK_END)
            self.buffer.write(chunk["PayloadPart"]["Bytes"])

In [82]:
import time

In [93]:
resp['Body'].read()

AttributeError: 'EventStream' object has no attribute 'read'